<a href="https://colab.research.google.com/github/PikoAll/machine-learning-e-deep-learning/blob/main/RNN_sentiment_e_GenerazioneTesto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RNN,LSTM,TBPTT
Dati Sequenziali(importante l'ordine della sequenza)
Prevenire azioni nel breve tempo futuro
non sara presentato ma le RNN sono utili per creare dei chatBot

predizione del sentiment recensioni film, uso un csv relativo al cap 8

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')

df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


Creazione del dataset

In [ ]:
target=df.pop('sentiment')
print(target.head())

0    1
1    0
2    0
3    1
4    0
Name: sentiment, dtype: int64


In [ ]:
ds_raw=tf.data.Dataset.from_tensor_slices((df.values,target.values))  #dovrebbe essere la creazione del dataset in modalita tensorflow diciamo forse

for ex in ds_raw.take(3):
  tf.print(ex[0].numpy()[0][:50],ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


creo il dataset di test train e convalida

In [ ]:
tf.random.set_seed(1)
ds_raw=ds_raw.shuffle(50000,reshuffle_each_iteration=False)#mischiamo tutto il dataset 50000 poiche ha 50000 righe

ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

ora codifichiamo le parole in numeri, ciascuna parola avra un numero

In [ ]:
from collections import Counter

try:
    tokenizer = tfds.features.text.Tokenizer()
except AttributeError:
    tokenizer = tfds.deprecated.text.Tokenizer()

token_count=Counter()

for i in ds_raw_train:
  tokens=tokenizer.tokenize(i[0].numpy()[0])
  token_count.update(tokens)

print('dimensione vocabolo:',len(token_count))

dimensione vocabolo: 87007


creo un dizionario 

In [ ]:
## Step 3: encoding each unique token into integers

try:
    encoder = tfds.features.text.TokenTextEncoder(token_count)
except AttributeError:
    encoder = tfds.deprecated.text.TokenTextEncoder(token_count)

example_str = 'This is an example!'
encoder.encode(example_str)

[232, 9, 270, 1123]

In [ ]:
## Step 3-A: define the function for transformation

def encode(text_tensor, label):
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    return encoded_text, label

## Step 3-B: wrap the encode function to a TF Op.
def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label], 
                          Tout=(tf.int64, tf.int64))

In [ ]:
ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

tf.random.set_seed(1)
for example in ds_train.shuffle(1000).take(5):
    print('Sequence length:', example[0].shape)
    
example

Sequence length: (24,)
Sequence length: (179,)
Sequence length: (262,)
Sequence length: (535,)
Sequence length: (130,)


(<tf.Tensor: shape=(130,), dtype=int64, numpy=
 array([  579,  1296,    32,   425,    40,   763,  9267,    65,   280,
          308,     6,   481,   155,   473,     2,     3,   684,     9,
          781,   176,   959,   730,  3917,    67,  9905,    13,   277,
           24,    35,   371, 16368,     6,    14, 17231,    29,   187,
         1651,   489,   503,   480,   143,    32,   270,  5851,  2402,
           13,  3592,  3443,   425,  3313,   256,   257,  1577,   117,
            8,   698,   270,   564,    56,     8,    42,  7517,  2629,
          820,    25,    60,    79,   343,    32,   645,    14,   528,
          241,    32,  1980,     8,    56,     8,    42,  1364,   573,
         5183,    43,    12,  3870,    32,   312,   642,   251,  1401,
        17232,     8,   698,   257,   750,     2,     9,    76,   235,
            8,    42,   235,   840,   666,   258, 17233,   419,    32,
        17234,   585,   420,   840,    25,    40,    13,    14,   198,
          266,   623,   173,  

Finora abbiamo convertito sequanze di parole in numeri interi, ora bisogna settare una lunghezza uguale a tutte le frasi per poterli gestire meglio nel tensore

In [ ]:

ds_subset = ds_train.take(8)
for example in ds_subset:
    print('Individual size:', example[0].shape)

## batching the datasets
ds_batched = ds_subset.padded_batch(
    4, padded_shapes=([-1], []))

for batch in ds_batched:
    print('Batch dimension:', batch[0].shape)

Individual size: (119,)
Individual size: (688,)
Individual size: (308,)
Individual size: (204,)
Individual size: (326,)
Individual size: (240,)
Individual size: (127,)
Individual size: (453,)
Batch dimension: (4, 688)
Batch dimension: (4, 453)


divisione in mini-batch

In [ ]:
train_data = ds_train.padded_batch(
    32, padded_shapes=([-1],[]))

valid_data = ds_valid.padded_batch(
    32, padded_shapes=([-1],[]))

test_data = ds_test.padded_batch(
    32, padded_shapes=([-1],[]))

IMPLEMENTAZIONE SimpleRNN

In [ ]:

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Embedding(1000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          32000     
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 36,193
Trainable params: 36,193
Non-trainable params: 0
_________________________________________________________________


Varie alternative di modelli RNN

In [ ]:
'''

from tensorflow.keras.layers import LSTM


model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.summary()


from tensorflow.keras.layers import GRU

model = Sequential()
model.add(Embedding(10000, 32))
model.add(GRU(32, return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.summary()

'''

'\n\nfrom tensorflow.keras.layers import LSTM\n\n\nmodel = Sequential()\nmodel.add(Embedding(10000, 32))\nmodel.add(LSTM(32, return_sequences=True))\nmodel.add(LSTM(32))\nmodel.add(Dense(1))\nmodel.summary()\n\n\nfrom tensorflow.keras.layers import GRU\n\nmodel = Sequential()\nmodel.add(Embedding(10000, 32))\nmodel.add(GRU(32, return_sequences=True))\nmodel.add(GRU(32))\nmodel.add(Dense(1))\nmodel.summary()\n\n'

In [ ]:
embedding_dim = 20
vocab_size = len(token_count) + 2

tf.random.set_seed(1)

## build the model
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        name='embed-layer'),
    
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, name='lstm-layer'),
        name='bidir-lstm'), 

    tf.keras.layers.Dense(64, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

bi_lstm_model.summary()

#------------------------------------------------------------------
## compile and train:
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy'])

history = bi_lstm_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

## evaluate on the test data
test_results= bi_lstm_model.evaluate(test_data)
print('Test Acc.: {:.2f}%'.format(test_results[1]*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed-layer (Embedding)      (None, None, 20)          1740180   
_________________________________________________________________
bidir-lstm (Bidirectional)   (None, 128)               43520     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,792,021
Trainable params: 1,792,021
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 76s 110ms/step - loss: 0.5263 - accuracy: 0.7347 - val_loss: 0.4411 - val_accuracy: 0.8092
Epoch 2/10
625/625 [==============================] - 68s 109ms/step - loss: 0.2748 - accuracy: 0

KeyboardInterrupt: ignored

#Modellazione del linguaggio naturale Do in input un testo e lui me ne genera uno simile
parliamo di generazione di testo e non comprensione del linguaggio che si fara dopo

Ho scaricato in locale un libro in italiano in formato txt che ora carichero

In [ ]:
with open('/content/TestoRNN','r') as fp:
  text=fp.read()

print(text)

Chi esamina per la prima volta il complesso e meraviglioso sviluppo del pensiero scientifico nel corso della storia, è facilmente portato a fissare la sua attenzione sui nomi di alcuni grandissimi pensatori. Questi, sembra, hanno creato di colpo le singole scienze od alcune loro particolari discipline; questi, ad un determinato stadio di esse, le hanno, con la forza del loro genio, incamminate su vie nuove e maggiormente feconde. La storia della scienza, in questo modo, apparisce quasi una storia di eroi.
Un esame più approfondito, invece, e specialmente lo studio accurato della letteratura scientifica, distrugge ben presto questa visione, che potrei dire alla Carlyle. Gli eroi rimangono, è vero, ma apparisce e si mostra, determinante ed essenziale, il lavoro più modesto, spesse volte anonimo, della collettività. Quelli, gli eroi, ci appariscono allora piuttosto come indici del loro tempo, come fiumi fatti impetuosi e potenti dall’indispensabile concorso di numerosi rigagnoli, come men

In [ ]:
start_indx=text.find('Chi esamina per la prima volta il complesso e meraviglioso ')
end_indx=text.find('Opuscules physiques et chymiques. I. Paris 1774, xxx, 360 pagine. In 8.')

text=text[start_indx:end_indx]
char_set=set(text)
print('lunghezza totale:',len(text))
print('caratteri unici',len(char_set))

lunghezza totale: 98305
caratteri unici 110


Convertiamo il testo in valori numerici

In [ ]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i,ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32)

print('Text encoded shape: ', text_encoded.shape)

print(text[:15], '     == Encoding ==> ', text_encoded[:15])
print(text_encoded[15:21], ' == Reverse  ==> ', ''.join(char_array[text_encoded[15:21]]))

Text encoded shape:  (98305,)
Chi esamina per      == Encoding ==>  [25 54 55  1 51 65 47 59 55 60 47  1 62 51 64]
[ 1 58 47  1 62 64]  == Reverse  ==>   la pr


Creazione del dataset tensorfòlow

In [ ]:
import tensorflow as tf


ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

25 -> C
54 -> h
55 -> i
1 ->  
51 -> e


Ulteriore preelaborazione

In [ ]:
seq_length = 40
chunk_size = seq_length + 1

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

## inspection:
for seq in ds_chunks.take(1):
    input_seq = seq[:seq_length].numpy()
    target = seq[seq_length].numpy()
    print(input_seq, ' -> ', target)
    print(repr(''.join(char_array[input_seq])), 
          ' -> ', repr(''.join(char_array[target])))

[25 54 55  1 51 65 47 59 55 60 47  1 62 51 64  1 58 47  1 62 64 55 59 47
  1 68 61 58 66 47  1 55 58  1 49 61 59 62 58 51]  ->  65
'Chi esamina per la prima volta il comple'  ->  's'


In [ ]:
## define the function for splitting x & y
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

## inspection:
for example in ds_sequences.take(2):
    print(' Input (x):', repr(''.join(char_array[example[0].numpy()])))
    print('Target (y):', repr(''.join(char_array[example[1].numpy()])))
    print()

 Input (x): 'Chi esamina per la prima volta il comple'
Target (y): 'hi esamina per la prima volta il comples'

 Input (x): 'so e meraviglioso sviluppo del pensiero '
Target (y): 'o e meraviglioso sviluppo del pensiero s'



In [ ]:
# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

tf.random.set_seed(1)
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)# drop_remainder=True)

ds

<BatchDataset shapes: ((None, 40), (None, 40)), types: (tf.int32, tf.int32)>

Costruzione del modello

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(
            rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

model = build_model(
    vocab_size = charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 256)         28160     
_________________________________________________________________
lstm (LSTM)                  (None, None, 512)         1574912   
_________________________________________________________________
dense_3 (Dense)              (None, None, 110)         56430     
Total params: 1,659,502
Trainable params: 1,659,502
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True
    ))

model.fit(ds, epochs=20)

Epoch 1/20
38/38 [==============================] - 2s 12ms/step - loss: 3.3346
Epoch 2/20
38/38 [==============================] - 1s 11ms/step - loss: 2.8237
Epoch 3/20
38/38 [==============================] - 1s 11ms/step - loss: 2.5471
Epoch 4/20
38/38 [==============================] - 1s 11ms/step - loss: 2.3851
Epoch 5/20
38/38 [==============================] - 1s 12ms/step - loss: 2.2791
Epoch 6/20
38/38 [==============================] - 1s 11ms/step - loss: 2.2052
Epoch 7/20
38/38 [==============================] - 1s 11ms/step - loss: 2.1521
Epoch 8/20
38/38 [==============================] - 1s 11ms/step - loss: 2.1038
Epoch 9/20
38/38 [==============================] - 1s 12ms/step - loss: 2.0604
Epoch 10/20
38/38 [==============================] - 1s 12ms/step - loss: 2.0192
Epoch 11/20
38/38 [==============================] - 1s 11ms/step - loss: 1.9807
Epoch 12/20
38/38 [==============================] - 1s 11ms/step - loss: 1.9475
Epoch 13/20
38/38 [==================

#--------------------------------------

In [ ]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 1.0]]
print('Probabilities:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())

Probabilities: [0.33333334 0.33333334 0.33333334]
array([[1, 2, 0, 1, 0, 1, 1, 2, 1, 1]])


In [ ]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 3.0]]
print('Probabilities:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())

Probabilities: [0.10650698 0.10650698 0.78698605]
array([[2, 2, 0, 2, 2, 2, 2, 2, 1, 2]])


Ora creero una metodo dove alla fine passero alcune parole e mi restituire un testo

In [ ]:
def sample(model, starting_str, 
           len_generated_text=500, 
           max_input_length=40,
           scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(
            scaled_logits, num_samples=1)
        
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()    

        generated_str += str(char_array[new_char_indx])
        
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str

tf.random.set_seed(1)
print(sample(model, starting_str='La storia'))#--------------------------------------------------- passo la parola in starting_str

La storiarda, e finomerà saltamente dal Mousto (uanza, spedeno a preitora della mendia dagio ancerantoresier er scoppossomò e difertare alla stano, carento il teriua, a recalcino la quana davororia dell’aria, e allora he proptato è chi ule carca e meruria riberitati del bragli e chimica, senzza stocera, e quingi che quel oporidi il favonia, vore Opera in un sulluzione penalda apteovi di Strichi, e ri dal esperia serò andosch quall’iononza chimie dotta bhinuzioni sifuramente vondese sovanne e rivoro i sir


In [ ]:
print(sample(model, starting_str='amico'))

amico della quasta vandori di avenzi in lavoro, questa di aveva il ritincò l’ait uns fiche, il respesso predere in quale Lavoisier, svenrantiche, sosso non già edo, da solto una sposse a viemba di chimico; na Ra le parti desto sia fogico, e degli attanzionere, mporione citati (1783 1775.
Quest detà che il qualla del Lovoisier e floria supintoità l’suogli.
Olara, e speditale cortere chemelte (1 sigini e storgata, ill’Accoid allunque dell’aritro acintice all’rotatti, il mogoro ben palti spessista compl
